In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# 1. Load Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize and reshape to (batch, height, width, channels)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
X_train = np.expand_dims(X_train, axis=-1)  # shape: (60000, 28, 28, 1)
X_test = np.expand_dims(X_test, axis=-1)

# One-hot encode labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# 2. Load baseline model
baseline_model = load_model('baseline_model2.h5')
print("✅ Baseline model loaded")

✅ Baseline model loaded


In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_ptq = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpdtjr47su/assets


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpdtjr47su/assets
2025-06-05 21:04:24.461823: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-06-05 21:04:24.461971: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-06-05 21:04:24.463768: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpdtjr47su
2025-06-05 21:04:24.464975: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-06-05 21:04:24.464984: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpdtjr47su
2025-06-05 21:04:24.468191: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-06-05 21:04:24.470058: I tensorflow/cc/saved_model/load

In [7]:
from pathlib import Path
import pathlib
# 3. Save the quantized TFLite model

tflite_models_dir = pathlib.Path('/Users/oscarpatrikminj/Documents/IITR/FMNIST/tflite_models')
tflite_models_dir.mkdir(exist_ok=True, parents=True)
tflite_model_file = tflite_models_dir/'model_ptq.tflite'
tflite_model_file.write_bytes(tflite_model_ptq)

188672

In [10]:
import os
# Load the quantized TFLite model
tflite_model_file = 'tflite_models/model_ptq.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

pred_list = []

for image in X_test:
    input_data = np.array(image, dtype=np.float32)
    input_data = input_data.reshape(1, input_data.shape[0], input_data.shape[1], 1)
    interpreter.set_tensor(input_index, input_data)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_index)
    prediction = np.argmax(prediction)
    pred_list.append(prediction)

accurate_count = 0
for idx in range(len(pred_list)):
    if pred_list[idx] == np.argmax(y_test[idx]):
        accurate_count += 1

accuracy = accurate_count / len(pred_list)
print('Accuracy:', accuracy)

# ---------- Model Size (Unzipped, in KB) ----------
size_kb = os.path.getsize(tflite_model_file) / 1024
print(f"📦 Model size (uncompressed): {size_kb:.2f} KB")





Accuracy: 0.9141
📦 Model size (uncompressed): 184.25 KB
